In [ ]:
# # Associate Ca2+ signal with sleep stages for each session & subsessions using crossregistration

#######################################################################################
                            # Define Experiment type #
#######################################################################################

DrugExperiment=0 # =1 if CGP Experiment // DrugExperiment=0 if Baseline Experiment

AnalysisID='' 

saveexcel=0

#######################################################################################
                                # Load packages #
#######################################################################################

import os
import numpy as np
from scipy import signal
import quantities as pq
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import sys 
from datetime import datetime
import shutil
from ast import literal_eval
from scipy.signal import find_peaks
from scipy.stats import zscore
from scipy import stats
from itertools import groupby
from IPython.display import display
from scipy.interpolate import griddata
import numpy as np
from scipy.signal import butter, filtfilt, hilbert
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy.matlib
from sklearn.decomposition import PCA
from scipy import stats
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample
from scipy.signal import resample_poly
from math import gcd
import warnings
warnings.filterwarnings("ignore")
import sys
class Tee:
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush()
    def flush(self):
        for f in self.files:
            f.flush()


minian_path = os.path.join(os.path.abspath('..'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))
sys.path.append(minian_path)

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)


In [ ]:
dir= "//10.69.168.1/crnldata/forgetting/Clementine/CheeseboardExperiment/Juin/YL/"

In [ ]:
data = {}
for dpath in Path(dir).glob('**/mappingsAB.pkl'):

    mappfile = open(dpath.parents[0]/ f'mappingsAB.pkl', 'rb')
    mapping = pickle.load(mappfile)
    mapping_sess = mapping['session']   

    mice = dpath.parents[0].parts[-1]
    NeuronType = dpath.parents[1].parts[-1]
    
    print(f"####################################################################################")
    print(f"################################### {mice} ####################################")
    print(f"####################################################################################")

    nb_minian_total=0
    dict_Calcium = {}
    dict_TodropFile = {}

    minian_folders = [f for f in dpath.parents[0].rglob('minian') if f.is_dir()]

    for minianpath in minian_folders: # for each minian folders found in this mouse 

        session=minianpath.parents[1].name 
        print(f'Session = {session}')
        session_path=minianpath.parents[2]

        session_date= minianpath.parents[2].name
        session_time= minianpath.parents[1].name
    
        minian_ds = open_minian(minianpath)
        dict_Calcium[session] = minian_ds['C'] # calcium traces 
        
        with open(minianpath.parent / f'TodropFileAB.json', 'r') as f:
            unit_to_drop = json.load(f)
            dict_TodropFile[session]  = unit_to_drop

        nb_minian_total+=1

        #######################################################################################
        # Distribute Ca2+ intensity & spikes to vigilance states for each sessions #
        #######################################################################################

        # Remove bad units from recordings

        C=dict_Calcium[session]
        unit_to_drop=dict_TodropFile[session]    

        C_sel=C.drop_sel(unit_id=unit_to_drop)

        CalciumSub = pd.DataFrame(C_sel, index=C_sel['unit_id'])

        indexMappList=mapping_sess[session]
        kept_uniq_unit_List=[]
        group_size_List=[]
        mapping['groupsize']=[len(mapping['group'].loc[i].sum()) for i in np.arange(len(mapping['group']))]
        for unit in CalciumSub.index:
            indexMapp = np.where(indexMappList == unit)[0]
            kept_uniq_unit_List.append(str(indexMapp))
            group_size_List.append(mapping['groupsize'].loc[indexMapp].values.tolist()[0])

        nb_unit=len(CalciumSub)
        print(f'...{nb_unit}/{len(C)} units kept in the session')

        unique, counts = numpy.unique(group_size_List, return_counts=True)
        print(f'...{counts[0]} units appears in {unique[0]} sessions, {counts[1]} units appears in {unique[1]} sessions, {counts[2]} units appears in {unique[2]} sessions, {counts[3]} units appears in {unique[3]} sessions')

        if nb_unit==0:
            print(f'no cells kept in the session')
            continue  # next iteration


mapping['group size']=[len(mapping['group'].loc[i].sum()) for i in np.arange(len(mapping['group']))]
mapping2=mapping.copy()
mapping2=mapping2[mapping2['group size']>1]
print(f'{len(mapping2)}/{len(mapping)} units appears in at least 2 sessions')